In [94]:
#!pip install tensorflow
#!pip install pandas as pd
#!pip install numpy as np
#!pip install --upgrade pip

In [95]:
import pandas as pd
import numpy as np

In [96]:
df = pd.read_csv("hero_selection_final.csv")

In [97]:
df1 = pd.read_csv("hero_names.csv")

In [98]:
matrix2 =np.array(df1.loc[:,['hero_id', 'Strength','Agility','Intelligence']])

In [100]:
df2 = pd.read_csv("hero_selection.csv")
#df2

In [8]:
matrix3 =np.array(df2.loc[:,['target']])

In [101]:
'''Category prediction of hero5 will contain the data of hero1, hero2, hero3, hero4.'''
matrix4 =np.array(df2.loc[:,[ 'Hero1','Hero2','Hero3','Hero4']])
matrix5 = np.array(df2.loc[:,[ 'Hero5']])
'''Category prediction of hero2 will contain the data of hero1.'''
matrix43 =np.array(df2.loc[:,[ 'Hero1']])
matrix53 = np.array(df2.loc[:,[ 'Hero2']])
'''Category prediction of hero3 will contain the data of hero1, hero2.'''
matrix42 =np.array(df2.loc[:,[ 'Hero1','Hero2']])
matrix52 = np.array(df2.loc[:,[ 'Hero3']])
'''Category prediction of hero4 will contain the data of hero1, hero2, hero3'''
matrix41 =np.array(df2.loc[:,[ 'Hero1','Hero2','Hero3']])
matrix51 = np.array(df2.loc[:,[ 'Hero4']])

In [107]:
''' target variable contains the category of next hero picked in the next pick. '''
def test_train(matrix4, matrix5):
    feature_matrixes = matrix4
    target_features = matrix5
    train_data = feature_matrixes[0:40000]
    test_data = feature_matrixes[40000:50000]
    train_target = target_features[0:40000]
    test_target = target_features[40000:50000]
    
    #train target preparation for categorical data
    train_target.reshape(40000,)
    final_index=[]
    for i in range(40000):
        hero = int(train_target[i][0])
        indexed = matrix2[hero-1]
      
        if indexed[1]==1:
            final_index.append(1)
        elif indexed[2]==1:
            final_index.append(2)
        elif indexed[3]==1:
            final_index.append(3)
    final_index = np.array(final_index)
    
    #test target preparation for categorical data
    test_target.reshape(10000,)
    final_index_test=[]
    for i in range(10000):
        hero = int(test_target[i][0])
        indexed = matrix2[hero-1]  
        if indexed[1]==1:
            final_index_test.append(1)
        elif indexed[2]==1:
            final_index_test.append(2)
        elif indexed[3]==1:
            final_index_test.append(3)
    final_index_test = np.array(final_index_test)
    
    return train_data, final_index, test_data, final_index_test

In [108]:
train_data5, final_index5, test_data5, final_index_test5 = test_train(matrix4[0:50000], matrix5[0:50000])
train_data4, final_index4, test_data4, final_index_test4 = test_train(matrix41[0:50000], matrix51[0:50000])
train_data3, final_index3, test_data3, final_index_test3 = test_train(matrix42[0:50000], matrix52[0:50000])
train_data2, final_index2, test_data2, final_index_test2 = test_train(matrix43[0:50000], matrix53[0:50000])


In [109]:
import tensorflow as tf
#import keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [113]:
'''
    Started the neural networks implementation with an Embedding layer and they are followed by 2 layers of 
    bidirectional LSTMs followed by a dense layer with activation method softmax.
'''
# Build the model.
inputs5 = keras.Input(shape=(4,), dtype="int32")
x5 = layers.Embedding(40000, 4)(inputs5)
x5 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x5)
x5 = layers.Bidirectional(layers.LSTM(50))(x5)
outputs5 = layers.Dense(4, activation="softmax")(x5)
model5 = keras.Model(inputs5, outputs5)
model5.summary()


inputs4 = keras.Input(shape=(3,), dtype="int32")
x4 = layers.Embedding(40000, 4)(inputs4)
x4 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x4)
x4 = layers.Bidirectional(layers.LSTM(50))(x4)
outputs4 = layers.Dense(4, activation="softmax")(x4)
model4 = keras.Model(inputs4, outputs4)
model4.summary()


inputs3 = keras.Input(shape=(2,), dtype="int32")
x3 = layers.Embedding(40000, 4)(inputs3)
x3 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x3)
x3 = layers.Bidirectional(layers.LSTM(50))(x3)
outputs3 = layers.Dense(4, activation="softmax")(x3)
model3 = keras.Model(inputs3, outputs3)
model3.summary()


inputs2 = keras.Input(shape=(1,), dtype="int32")
x2 = layers.Embedding(40000, 4)(inputs2)
x2 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x2)
x2 = layers.Bidirectional(layers.LSTM(50))(x2)
outputs2 = layers.Dense(4, activation="softmax")(x2)
model2 = keras.Model(inputs2, outputs2)
model2.summary()



Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 4, 4)              160000    
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 4, 100)            22000     
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 100)               60400     
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 404       
Total params: 242,804
Trainable params: 242,804
Non-trainable params: 0
_________________________________________________________________
Model: "functional_20"
_________________________________________________________________
Layer (type)            

In [114]:
# Compile the model.
model5.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model4.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model3.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model2.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [116]:
#model.fit(train_data, train_target, batch_size=30, epochs=100)
model5.fit(
  train_data5,
  to_categorical(final_index5),
  epochs=20,
  batch_size=32
)

model4.fit(
  train_data4,
  to_categorical(final_index4),
  epochs=20,
  batch_size=32
)


model3.fit(
  train_data3,
  to_categorical(final_index3),
  epochs=20,
  batch_size=32
)

model2.fit(
  train_data2,
  to_categorical(final_index2),
  epochs=20,
  batch_size=32
)

Epoch 1/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9847 - accuracy: 0.5038
Epoch 2/20
1250/1250 [==============================] - 4s 4ms/step - loss: 0.9841 - accuracy: 0.5053
Epoch 3/20
1250/1250 [==============================] - 4s 4ms/step - loss: 0.9829 - accuracy: 0.5060
Epoch 4/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9819 - accuracy: 0.5065
Epoch 5/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9806 - accuracy: 0.5085
Epoch 6/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9795 - accuracy: 0.5086
Epoch 7/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9780 - accuracy: 0.5097
Epoch 8/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9761 - accuracy: 0.5104
Epoch 9/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9741 - accuracy: 0.5129
Epoch 10/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.971

1250/1250 [==============================] - 3s 2ms/step - loss: 0.9654 - accuracy: 0.5417


In [119]:
model5.evaluate(
  test_data5,
  to_categorical(final_index_test5)
)

model4.evaluate(
  test_data4,
  to_categorical(final_index_test4)
)

model3.evaluate(
  test_data3,
  to_categorical(final_index_test3)
)

model2.evaluate(
  test_data2,
  to_categorical(final_index_test2)
)

313/313 [==============================] - 0s 887us/step - loss: 0.9778 - accuracy: 0.5386


[0.977778434753418, 0.5386000275611877]

In [120]:
predictions5 = model5.predict(test_data5)
predictions4 = model4.predict(test_data4)
predictions3 = model3.predict(test_data3)
predictions2 = model2.predict(test_data2)

In [92]:
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

[2 2 3 3 2 1 2 2 3 1]


In [127]:
y_pred5 = np.argmax(predictions5, axis=1)
y_pred4 = np.argmax(predictions4, axis=1)
y_pred3 = np.argmax(predictions3, axis=1)
y_pred2 = np.argmax(predictions2, axis=1)
y_pred5[0:20]

array([2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 3, 2, 2, 1, 3, 3, 1, 3])

In [126]:
'''Self test is used. final_index_test5 contains the test target for 5th hero.
    Similarly, final_index_test4 is for 4th hero, final_index_test3 is for 3rd hero, 
    final_index_test4 is for 2nd hero.
'''
my_ytest5 = np.array(final_index_test5)
my_ytest4 = np.array(final_index_test4)
my_ytest3 = np.array(final_index_test3)
my_ytest2 = np.array(final_index_test2)

my_ytest_finalhero5 = my_ytest5
my_ytest_finalhero4 = my_ytest4
my_ytest_finalhero3 = my_ytest3
my_ytest_finalhero2 = my_ytest2

#my_ytest_finalhero
my_ypred5 = np.array(y_pred5)
my_ypred4 = np.array(y_pred4)
my_ypred3 = np.array(y_pred3)
my_ypred2 = np.array(y_pred2)

my_ypred_finalhero5 = my_ypred5
my_ypred_finalhero4 = my_ypred4
my_ypred_finalhero3 = my_ypred3
my_ypred_finalhero2 = my_ypred2

#my_ypred_finalhero
totals = my_ytest5.shape[0]
#totals
num_correct5 = sum(p == t for p, t in zip(my_ypred_finalhero5, my_ytest_finalhero5))
num_correct4 = sum(p == t for p, t in zip(my_ypred_finalhero4, my_ytest_finalhero4))
num_correct3 = sum(p == t for p, t in zip(my_ypred_finalhero3, my_ytest_finalhero3))
num_correct2 = sum(p == t for p, t in zip(my_ypred_finalhero2, my_ytest_finalhero2))

#num_correct
total5 = (num_correct5/totals)*100
total4 = (num_correct4/totals)*100
total3 = (num_correct3/totals)*100
total2 = (num_correct2/totals)*100

print("Prediction accuracy for 5th hero:", total5)
print("Prediction accuracy for 4th hero:", total4)
print("Prediction accuracy for 3rd hero:", total3)
print("Prediction accuracy for 2nd hero:", total2)

Prediction accuracy for 5th hero: 46.300000000000004
Prediction accuracy for 4th hero: 47.53
Prediction accuracy for 3rd hero: 47.71
Prediction accuracy for 2nd hero: 53.86


In [93]:
print(final_index_test[10:20]) # [7, 2, 1, 0, 4]

[2 2 3 1 3 3 3 2 3 3]
